In [1]:
import numpy as np
import urllib.request
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import sklearn
import joblib
import itertools
import h5py
import hdf5storage
from torch.utils.data import DataLoader, Dataset
import seaborn as sns
from tsmoothie.utils_func import sim_randomwalk
from tsmoothie.smoother import LowessSmoother
from tsmoothie.utils_func import sim_seasonal_data
from tsmoothie.smoother import DecomposeSmoother
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from data_cul import add_label

plt.rcParams['font.sans-serif']=['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False  # 用来正常显示负号

In [2]:
path = '../data/MM_muti_0.mat'

mat_data = hdf5storage.loadmat(path)
mat_data.keys()
a = mat_data['MM_muti_F1_no_nor'][0]


In [3]:
def cul_data(data):
    """
    cul_data处理

    data:  原始数据

    return: 返回值是处理后的结果
    """
    ## 阵元归一
    divisor = data[0,:]

    divisor = divisor[np.newaxis,:]
    # print(divisor.shape)

    temp_data = data / divisor
    # 特征工程 
    ## tsmoothie 
    ## 修改行数
    # sm_data = temp_data[1000:2001, :].T
    sm_data = temp_data

    # operate smoothing
    #参数整定
    smoother = DecomposeSmoother(smooth_type='lowess', periods=5,
                                smooth_fraction=0.2)
    smoother.smooth(sm_data)

    low, up = smoother.get_intervals('sigma_interval')
    mean_data = np.mean(smoother.smooth_data, axis=1)
    return smoother.smooth_data, mean_data # 处理后的数据
    # return sm_data # 原始数据
    

In [4]:
path = ['../data/MM_muti_0.mat', '../data/MM_muti_04.mat', '../data/MM_muti_07.mat']
all_data_test = []
for i in range(len(path)):
    mat_data = hdf5storage.loadmat(path[i])
    a = mat_data['MM_muti_F1_no_nor'][0]
    b = []
    for k in range(len(a)):
        _,temp_data = cul_data(a[k])
        b.append(temp_data)
    all_data_test.append(add_label(np.array(b), i))

In [5]:
data_01 = all_data_test[0]
data_02 = all_data_test[1]
data_03 = all_data_test[2]

In [6]:
# path = ['../data/MM_muti_0.mat', '../data/MM_muti_04.mat', '../data/MM_muti_07.mat']
# all_data_test = []
# for i in path:
#     mat_data = hdf5storage.loadmat(i)
#     temp_data = mat_data['MM_muti_F1_no_nor'][0]

In [9]:
Features = data_02[:-1, :].T
Labels = data_02[-1,:].T

X_test =Features
y_test = Labels

rcf = joblib.load("./model/rfc_3_model-0.937.model")

y_pred = rcf.predict(X_test)

accuracy = np.mean(y_pred == y_test)
print("Accuracy:{:.3f}".format(accuracy*100))
print(y_pred[1000:2000])

Accuracy:0.000
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.